In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

# Define the hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10

In [2]:
from graph_bridges import data_path

In [3]:
raw_dir = os.path.join(data_path,"raw")

In [4]:

# MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root=raw_dir,
                                           train=True,
                                           transform=transform,
                                           download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)


In [25]:
import torch
import torch.nn as nn

# Define the Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, in_channels, latent_dim):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1),  # Preserve input shape
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Preserve input shape
            nn.ReLU(),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),  # Preserve input shape
            nn.ReLU(),
            nn.Conv2d(32, in_channels, kernel_size=3, stride=1, padding=1),  # Preserve input shape
            nn.Sigmoid(),  # Use sigmoid to ensure values are in [0, 1] for images
        )

    def forward(self, x):
        # Encoder
        x = self.encoder(x)

        # Decoder
        x = self.decoder(x)

        return x

In [34]:
28*28

784

In [33]:
size_ = 17
in_channels = 1
latent_dim = 20
model = Autoencoder(in_channels,latent_dim)
model(torch.rand(64,in_channels,size_,size_)).shape

torch.Size([64, 1, 17, 17])

In [ ]:

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the autoencoder
for epoch in range(num_epochs):
    for data in train_loader:
        inputs, _ = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'autoencoder.pth')


Epoch [1/10], Loss: 0.0014


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable

# Define hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10
latent_dim = 20  # Dimensionality of the latent space

# MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transform,
                               download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

# Define the VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            nn.Flatten(),
        )

        self.fc_mu = nn.Linear(64 * 7 * 7, latent_dim)
        self.fc_logvar = nn.Linear(64 * 7 * 7, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64 * 7 * 7),
            nn.ReLU(),
            nn.Unflatten(1, (64, 7, 7)),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Sigmoid(),
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

# Instantiate the VAE model
model = VAE()

# Loss function and optimizer
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the VAE
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss / len(train_loader.dataset):.4f}')

# Generating samples from the VAE
model.eval()
with torch.no_grad():
    z_sample = torch.randn(16, latent_dim)
    sample = model.decoder(z_sample)
    torchvision.utils.save_image(sample.view(16, 1, 28, 28), 'sample_vae.png')

# Save the trained VAE model
torch.save(model.state_dict(), 'vae.pth')


Epoch [1/10], Loss: 143.8125
Epoch [2/10], Loss: 110.0067
Epoch [3/10], Loss: 106.9124
Epoch [4/10], Loss: 105.3339
Epoch [5/10], Loss: 104.3399
Epoch [6/10], Loss: 103.5846
Epoch [7/10], Loss: 103.0301
Epoch [8/10], Loss: 102.5053
Epoch [9/10], Loss: 102.1958
Epoch [10/10], Loss: 101.8511
